In [1]:
class Rule:
    def __init__(self, numRules):
        self.number = numRules
        self.the_guard = Guard(self)
        self.count = 0
        self.index = 0

    def first(self):
        return self.the_guard.n

    def last(self):
        return self.the_guard.p

    def get_rules(self):
        reglas = {}
        rules = []
        processedRules = 0
        text = ''
        charCounter = 0
        text += "Usage\tRule\n"
        reglas["reglas"] = "Rule"
        rules.append(self)

        while (processedRules < len(rules)):
            currentRule = rules[processedRules]
            text += " " + str(currentRule.count) + '\tR' + str(processedRules) + ' -> '
            sym = currentRule.first()
            temporal = []
            while True:
                if sym.is_guard():
                    break
                if sym.is_nonterminal():
                    rule = sym.r
                    if len(rules) > rule.index and rules[rule.index] == rule:
                        index = rule.index
                    else:
                        index = len(rules)
                        rule.index = index
                        rules.append(rule)
                    text += 'R' + str(index)
                    temporal.append('R' + str(index))
                else:
                    if sym.value == ' ':
                        text += '_'
                        temporal.append('_')
                    elif sym.value == '\n':
                        text += '\\n'
                        temporal.append('\\n')
                    else:
                        text += str(sym.value)
                        temporal.append(str(sym.value))
                text += ' '
                sym = sym.n
            reglas['R' + str(processedRules) ] = temporal
            text += '\n'
            processedRules += 1
        return text,reglas


In [2]:
class Symbol:
    def __init__(self):
        self.num_terminals = 100000
        self.prime = 2265539
        self.value=' '
        global digrams

    def clone(self):
        sym = Symbol()
        sym.value = self.value
        sym.n = self.n
        sym.p = self.p
        return sym

    def join(self, left, right):
        if left.n != None:
            left.delete_digram()
            if (right.p != None and right.n != None and right.value == right.p.value and right.value == right.n.value):
                digrams[str(right.value) + str(right.n.value)] = right
            if (left.p != None and left.n != None and left.value == left.p.value and left.value == left.n.value):
                digrams[str(left.p.value) + str(left.value)] = left.p
        left.n = right
        right.p = left

    def insert_after(self, to_insert):
        self.join(to_insert, self.n)
        self.join(self, to_insert)

    def digram(self):
        return str(self.value) + str(self.n.value)

    def delete_digram(self):
        if self.n.is_guard(): return

        if self.digram() in digrams:
            dummy = digrams[self.digram()]
            if dummy == self:
                del digrams[self.digram()]

    def is_guard(self):
        return False

    def is_nonterminal(self):
        return False

    def check(self):
        if self.n.is_guard():
            return False

        if self.digram() not in digrams:
            digrams[self.digram()] = self
            return False

        found = digrams[self.digram()]
        if found.n != self:
            self.match(self, found)
        return True

    def cleanup(self):
        pass

    def substitute(self, r):
        self.cleanup()
        self.n.cleanup()
        self.p.insert_after(NonTerminal(r))
        if not self.p.check(): self.p.n.check()

    def match(self, newD, matching):
        #print 'Matching ', newD.value, matching.value
        global numRules

        if matching.p.is_guard() and matching.n.n.is_guard():
            #print 'Using Rule'
            r = (matching.p).r
            newD.substitute(r)
        else:
            #print 'Creating Rule'
            r = Rule(numRules)
            numRules += 1
            first = newD.clone()
            second = newD.n.clone()
            r.the_guard.n = first
            first.p = r.the_guard
            first.n = second
            second.p = first
            second.n = r.the_guard
            r.the_guard.p = second
            matching.substitute(r)
            newD.substitute(r)
            digrams[str(first.value)+str(first.n.value)]=first

        #print 'Checking Rule Utility'
        if (r.first()).is_nonterminal() and (r.first()).r.count == 1:
            (r.first()).expand()

    def hash_code(self):
        code = 21599 * self.value + 20507*self.n.value
        code = code%self.prime
        return code

    def equals(self,obj):
        return self.value == obj.value and self.n.value == obj.n.value

In [3]:
class Terminal(Symbol):
    def __init__(self, value):
        self.value=value
        self.p = None
        self.n = None

    def cleanup(self):
        self.join(self.p, self.n)
        self.delete_digram()

    def clone(self):
        sym = Terminal(self.value)
        sym.p = self.p
        sym.n = self.n
        return sym



In [4]:
class NonTerminal(Symbol):
    def __init__(self, rule):
        self.r = rule
        self.r.count += 1
        #self.value=self.numTerminals+self.r.number
        self.value = self.r.number
        self.p = None
        self.n = None

    def clone(self):
        sym = NonTerminal(self.r)
        sym.p = self.p
        sym.n = self.n
        return sym

    def cleanup(self):
        self.join(self.p, self.n)
        self.delete_digram()
        self.r.count -= 1

    def is_nonterminal(self):
        return True

    def expand(self):
        self.join(self.p, self.r.first())
        self.join(self.r.last(), self.n)
        digrams[str(self.r.last().value) + str(self.r.last().n.value)] = self.r.last()
        self.r.the_guard.r = None
        self.r.the_guard = None

In [5]:

class Guard(Symbol):
    def __init__(self, rule):
        self.r = rule
        self.value = ' '
        self.p = self
        self.n = self

    def cleanup(self):
        self.join(self.p, self.n)

    def is_guard(self):
        return True

    def check(self):
        return False

In [6]:
digrams = {}

numRules=0

def run_sequitur(text):
    global numRules
    first_rule = Rule(numRules)
    numRules += 1

    global digrams
    digrams={}

    for c in text:
        (first_rule.last()).insert_after(Terminal(c))
        first_rule.last().p.check()    

    res = first_rule.get_rules()
    #print(res[0])
    #print(res[1])
    return res[1]



In [7]:
texto = 'bbebeebebeeabcda$$cabb_bebe!!e!!bebebbe$$be_eabcdacabbbe_beebebebbe$$beeabcdacabbbebeebebebbe_eeabcdacabbbebeebebebbebeeabaccdacabbbebeebebebbebeeabcdacabbbebeebebebbebeeabcdacabbbebeebebebbebeeabcdacab'
texto2 = ("C1C12C2C12C19C4C10C19C12C15C12C18C9C2C11C19C5C8C5C18C4C17C4C11C11C2C5C13C6C1C13C20C8C13C6C12C16C7C5C15C6C3C5C18C9C11C14C0C13C16C15C5C16C14C18C6C6C6C1C17C14C1C0C0C11C7C20C10C12C18C14C19C2C5C6C18C14C7C19C19C5C4C13C3C9C2C11C11C12C12C7C15C20C19C9C19C2C14C7C20C8C15C4C2C13C13C17C6C17C13C18C13C14C4C19C11C11C3C14C14C19C16C18C2C13C2C6C19C4C7C16C8C16C0C3C12C17C8C6C6C14C9C4C19C20C1C2C10C18C3C3C12C0C2C17C12C14C16C6C7C11C5C10C9C17C2C6C8C19C1C7C4C3C3C13C11C12C13C3C14C2C4C16C8C14C20C4C0C2C13C18C18C9C19C11C2C1C5C3C16C19C9C8C5C1C14C7C1C8C7C11C4C3C4C2C2C12C5C15C16C19C5C6C3C18C19C20C14C1C16C20C7C17C6C9C10C9C15C19C19C7C15C9C2C6C2C19C20C7C14C10C8C19")

run_sequitur(texto2)

{'reglas': 'Rule',
 'R0': ['R1',
  'R2',
  'R3',
  'R4',
  'R5',
  'R5',
  'R6',
  'R7',
  'R8',
  'R9',
  'R8',
  'R10',
  '7',
  'R10',
  'R11',
  'R12',
  'R13',
  'R14',
  'R1',
  'R15',
  'R1',
  'R14',
  'R16',
  'R17',
  'R14',
  'R10',
  'R18',
  'R19',
  'R20',
  'R21',
  'R11',
  'R22',
  'R1',
  'R23',
  'R24',
  'R23',
  'R25',
  'R26',
  'R27',
  'R28',
  '7',
  'R29',
  'R30',
  'R30',
  'R28',
  'R14',
  'R31',
  'R4',
  'R32',
  'R29',
  'R9',
  'R21',
  'R33',
  'R34',
  'R25',
  'R18',
  'R13',
  'R35',
  'R9',
  'R7',
  'R11',
  'R2',
  'R14',
  'R36',
  'R31',
  'R37',
  'R9',
  'R38',
  'R34',
  'R15',
  'R36',
  'R39',
  'R1',
  'R1',
  'R40',
  'R41',
  'R42',
  'R1',
  'R36',
  'R25',
  'R11',
  '1',
  'R43',
  'R44',
  'R33',
  'R16',
  'R1',
  'R45',
  'R3',
  'R14',
  'R17',
  'R9',
  'R17',
  'R30',
  'R35',
  'R20',
  'R41',
  'R9',
  'R27',
  'R36',
  'R9',
  'R25',
  'R46',
  'R47',
  'R4',
  'R48',
  'R20',
  'R20',
  '0',
  'R12',
  'R41',
  'R20',
  'R

In [8]:
abc = run_sequitur(texto2)

In [9]:
# S: texto original
print('S -> ',abc['R0'])

S ->  ['R1', 'R2', 'R3', 'R4', 'R5', '0', 'R6', 'R7', '5', 'R2', 'R8', 'R9', 'R6', 'R10', 'R11', 'R8', 'R5', 'R12', 'R5', '1', 'R13', 'R14', 'R10', 'R15', 'R1', 'R16', 'R17', 'R15', 'R3', 'R18', 'R19', 'R20', 'R19', 'R21', 'R22', 'R23', 'R24', 'R25', 'R26', 'R27', 'R10', 'R19', 'R27', 'R28', 'R8', 'R29', 'R27', 'R1', 'R12', 'R24', 'R30', 'R25', 'R25', 'R23', 'R12', 'R31', 'R25', 'R7', 'R32', 'R33', 'R34', 'R27', 'R32', 'R12', 'R33', 'R33', 'R10', 'R28', 'R35', 'R9', 'R13', 'R7', 'R14', 'R12', 'R36', 'R31', 'R22', 'R22', 'R33', 'R3', 'R37', 'R17', 'R36', 'R28', 'R38', 'R26', 'R12', 'R27', 'R12', 'R26', 'R21', 'R26', 'R28', 'R39', 'R23', 'R23', 'R16', 'R24', 'R24', 'R40', 'R27', 'R21', 'R38', 'R14', 'R27', 'R22', 'R37', 'R41', 'R42', 'R25', 'R16', 'R7', 'R12', 'R21', 'R29', 'R24', 'R22', 'R39', 'R31', 'C', 'R3', 'R25', 'R8', 'R43', 'R44', 'R45', 'R12', 'R44', 'R24', 'R42', 'R18', 'R23', 'R19', 'R46', 'R12', 'R47', 'R21', 'R40', 'C', 'R12', 'R28', 'R43', 'R26', 'R30', 'R44', 'R35', 'R24',

In [10]:
# Reglas
for i in range(1,11):
    valor = 'R'+str(i)
    print(valor,' -> ',abc[valor])

R1  ->  ['C', '1']
R2  ->  ['R1', 'R14']
R3  ->  ['R14', '1']
R4  ->  ['R3', 'R22']
R5  ->  ['4', 'R1']
R6  ->  ['R1', 'R22']
R7  ->  ['1', 'R3']
R8  ->  ['1', 'R21']
R9  ->  ['R22', 'R3', '1']
R10  ->  ['5', 'C']
